In [30]:
from mdp import *

In [31]:
class MDP:

    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state."""

    def __init__(self, init, actlist, terminals, transitions = {}, reward = None, states=None, gamma=.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        if states:
            self.states = states
        else:
            ## collect states from transitions table
            self.states = self.get_states_from_transitions(transitions)
            
        
        self.init = init
        
        if isinstance(actlist, list):
            ## if actlist is a list, all states have the same actions
            self.actlist = actlist
        elif isinstance(actlist, dict):
            ## if actlist is a dict, different actions for each state
            self.actlist = actlist
        
        self.terminals = terminals
        self.transitions = transitions
        if self.transitions == {}:
            print("Warning: Transition table is empty.")
        self.gamma = gamma
        if reward:
            self.reward = reward
        else:
            self.reward = {s : 0 for s in self.states}
        #self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""
        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""
        if(self.transitions == {}):
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Set of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""
        if state in self.terminals:
            return [None]
        else:
            return self.actlist




In [32]:
states = [1,2,3,4,5,6,7,8,9,10,11]
terminals = [3,4,6,7,9,10,11]
nonterminal = [1,2,5,8]
prob = dict()
actions = dict([(s,[0]) for s in states])
actions[1] = [2]
actions[2] = [1,3,4,5]
actions[5] = [2,6,7,8]
actions[8] = [5,9,10,11]
print("Actions: ", actions)
rewards_terminal = [(3, 2), (4, 2), (6, 3), (7, 3), (9, -10), (10, -10), (11, 10)]
def T(state,action, p):
    prob[(state,action)] = p

T(1,2,[(1,0.1), (2,0.9)])
T(2,1,[(1, 0.7), (2, 0.1), (3, 0.1), (4, 0.1)])
T(2,3,[(1, 0.1), (2, 0.1), (3, 0.6), (4, 0.1), (5, 0.1)])
T(2,4,[(1, 0.1), (2, 0.1), (3, 0.1), (4, 0.6), (5, 0.1)])
T(2,5,[(1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.6)])
T(5,2,[(2, 0.6), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.1)])
T(5,6,[(2, 0.1), (5, 0.1), (6, 0.6), (7, 0.1), (8, 0.1)])
T(5,7,[(2, 0.1), (5, 0.1), (6, 0.1), (7, 0.6), (8, 0.1)])
T(5,8,[(2, 0.1), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.6)])
T(8,5,[(5, 0.6), (8, 0.1), (9, 0.1), (10, 0.1), (11, 0.1)])
T(8,9,[(5, 0.1), (8, 0.1), (9, 0.6), (10, 0.1), (11, 0.1)])
T(8,10,[(5, 0.1), (8, 0.1), (9, 0.1), (10, 0.6), (11, 0.1)])
T(8,11,[(5, 0.1), (8, 0.1), (9, 0.1), (10, 0.1), (11, 0.6)])
init = 1


Actions:  {1: [2], 2: [1, 3, 4, 5], 3: [0], 4: [0], 5: [2, 6, 7, 8], 6: [0], 7: [0], 8: [5, 9, 10, 11], 9: [0], 10: [0], 11: [0]}


In [33]:
print("States: ", states)
print("Terminal States: ", terminals)
print("Actions: ", actions)
print("Tranistion Matrix: ", prob)

States:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Terminal States:  [3, 4, 6, 7, 9, 10, 11]
Actions:  {1: [2], 2: [1, 3, 4, 5], 3: [0], 4: [0], 5: [2, 6, 7, 8], 6: [0], 7: [0], 8: [5, 9, 10, 11], 9: [0], 10: [0], 11: [0]}
Tranistion Matrix:  {(1, 2): [(1, 0.1), (2, 0.9)], (2, 1): [(1, 0.7), (2, 0.1), (3, 0.1), (4, 0.1)], (2, 3): [(1, 0.1), (2, 0.1), (3, 0.6), (4, 0.1), (5, 0.1)], (2, 4): [(1, 0.1), (2, 0.1), (3, 0.1), (4, 0.6), (5, 0.1)], (2, 5): [(1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.6)], (5, 2): [(2, 0.6), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.1)], (5, 6): [(2, 0.1), (5, 0.1), (6, 0.6), (7, 0.1), (8, 0.1)], (5, 7): [(2, 0.1), (5, 0.1), (6, 0.1), (7, 0.6), (8, 0.1)], (5, 8): [(2, 0.1), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.6)], (8, 5): [(5, 0.6), (8, 0.1), (9, 0.1), (10, 0.1), (11, 0.1)], (8, 9): [(5, 0.1), (8, 0.1), (9, 0.6), (10, 0.1), (11, 0.1)], (8, 10): [(5, 0.1), (8, 0.1), (9, 0.1), (10, 0.6), (11, 0.1)], (8, 11): [(5, 0.1), (8, 0.1), (9, 0.1), (10, 0.1), (11, 0.6)]}


In [34]:
def value_iteration(mdp, epsilon=0.001):
    
    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        for s in mdp.states:
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in CustomMDP.actions[s]])
            delta = max(delta, abs(U1[s] - U[s]))
        if delta < epsilon * (1 - gamma) / gamma:
            return U

In [35]:
def setR(reward):
    rewards_nonterminal = []
    for s in nonterminal:
        rewards_nonterminal.append((s,reward))
    rewards = rewards_terminal + rewards_nonterminal
    return rewards
            
def getR(state):
    for r in rewards:
        if r[0] == state:
            return r[1]


def Value(s,a,U):
    total = 0
    if a != 0:
        for p in prob[(s,a)]:
            total += p[1] * U[p[0]]
    return total

In [36]:
def value_iteration(gamma):
    "Solving an MDP by value iteration."
    epsilon=0.01
    U1 = dict([(s, 0) for s in states])
    count = 0
    while True:
        U = U1.copy()
        delta = 0
        for s in states:
            l = []
            for a in actions[s]:
                l.append(Value(s,a,U))
            if len(l) > 0:
                m = max(l)
            else:
                m = 0
            U1[s] = getR(s) + gamma*m
            U1[s] = round(U1[s],4)
            delta = max(delta, abs(U1[s] - U[s]))
        count += 1
        #print(U)
        if delta <= epsilon * (1 - gamma) / gamma:
            #print("Number of iterations: ", count-1)
            return U

In [37]:
reward = -0.1
print("Non-terminal reward: ", reward)
rewards = setR(reward)
print("Rewards: ", rewards)
gamma = 1
print("Discounting: ", gamma)
utility = value_iteration(gamma = 1)
print("Utility: ",utility)

Non-terminal reward:  -0.1
Rewards:  [(3, 2), (4, 2), (6, 3), (7, 3), (9, -10), (10, -10), (11, 10), (1, -0.1), (2, -0.1), (5, -0.1), (8, -0.1)]
Discounting:  1
Utility:  {1: 3.3516, 2: 3.4627, 3: 2, 4: 2, 5: 4.1355, 6: 3, 7: 3, 8: 4.7928, 9: -10, 10: -10, 11: 10}


In [43]:
reward = -0.7
print("Non-terminal reward: ", reward)
rewards = setR(reward)
print("Rewards: ", rewards)
gamma = 0.8
print("Discounting: ", gamma)
utility = value_iteration(gamma = 0.8)
print("Utility: ",utility)

Non-terminal reward:  -0.7
Rewards:  [(3, 2), (4, 2), (6, 3), (7, 3), (9, -10), (10, -10), (11, 10), (1, -0.7), (2, -0.7), (5, -0.7), (8, -0.7)]
Discounting:  0.8
Utility:  {1: -0.3354, 2: 0.5451, 3: 2.0, 4: 2.0, 5: 1.359, 6: 3.0, 7: 3.0, 8: 2.8355, 9: -10.0, 10: -10.0, 11: 10.0}


In [38]:
reward = -0.1
print("Non-terminal reward: ", reward)
rewards = setR(reward)
print("Rewards: ", rewards)
gamma = 0.8
print("Discounting: ", gamma)
utility = value_iteration(gamma = 0.8)
print("Utility: ",utility)

Non-terminal reward:  -0.1
Rewards:  [(3, 2), (4, 2), (6, 3), (7, 3), (9, -10), (10, -10), (11, 10), (1, -0.1), (2, -0.1), (5, -0.1), (8, -0.1)]
Discounting:  0.8
Utility:  {1: 1.1414, 2: 1.5999, 3: 2.0, 4: 2.0, 5: 2.4196, 6: 3.0, 7: 3.0, 8: 3.5799, 9: -10.0, 10: -10.0, 11: 10.0}


In [39]:
#pi = best_policy(mdp, value_iteration(mdp, .01))

In [40]:
def optimal_policy(U):
    pi = {}
    for s in states:
        l = []
        for a in actions[s]:
            l.append((a,Value(s,a,U)))
        if len(l) > 0 :
            pi[s] = max(l,key=lambda item:item[1])[0]
    return pi

In [41]:
def policy_evaluation(policy,U,gamma,k=20):
    for i in range(k):
        for s in states:
            U[s] = getR(s) + gamma*Value(s,policy[s],U)
    print(U)
    return U
    
def policy_iteration(policy,gamma):
    U = dict([(s, 0) for s in states])
    while True:
        U = policy_evaluation(policy,U,gamma)
        #print(U)
        unchanged = True
        for s in states:
            l = []
            for a in actions[s]:
                if a != 0: 
                    l.append((a,Value(s,a,U)))
            if len(l) > 0:
                m = max(l,key=lambda item:item[1])[1]
            if m > Value(s,policy[s],U):
                if len(l) > 0:
                    policy[s] = max(l,key=lambda item:item[1])[0]
                    unchanged = False
        if unchanged:
            #U = [round(U[s],4) for s in states]
            print(U)
            return policy 

In [42]:
best_policy = optimal_policy(utility)
print("Optimal Policy: ", best_policy)

print("\n Policy iteration")
policy ={1: 2, 2: 1, 5: 2, 8: 5, 3: 0, 4: 0, 6: 0, 7: 0, 9: 0, 10: 0, 11: 0}
print("Given Policy: ",policy)
p = policy_iteration(policy,gamma)
print("Optimal Policy: ",p)

Optimal Policy:  {1: 2, 2: 5, 3: 0, 4: 0, 5: 8, 6: 0, 7: 0, 8: 11, 9: 0, 10: 0, 11: 0}

 Policy iteration
Given Policy:  {1: 2, 2: 1, 5: 2, 8: 5, 3: 0, 4: 0, 6: 0, 7: 0, 9: 0, 10: 0, 11: 0}
{1: 0.14981111191040095, 2: 0.3303194191265836, 3: 2.0, 4: 2.0, 5: 0.5240949594599779, 6: 3.0, 7: 3.0, 8: -0.7048201563646231, 9: -10.0, 10: -10.0, 11: 10.0}
{1: 0.9711521606175738, 2: 1.3798055385670014, 3: 2.0, 4: 2.0, 5: 2.1466115329030124, 6: 3.0, 7: 3.0, 8: 3.5562270898176536, 9: -10.0, 10: -10.0, 11: 10.0}
{1: 1.1445153617245039, 2: 1.6013251844289011, 3: 2.0, 4: 2.0, 5: 2.4201207098687108, 6: 3.0, 7: 3.0, 8: 3.5800104965103188, 9: -10.0, 10: -10.0, 11: 10.0}
{1: 1.1445153617245039, 2: 1.6013251844289011, 3: 2.0, 4: 2.0, 5: 2.4201207098687108, 6: 3.0, 7: 3.0, 8: 3.5800104965103188, 9: -10.0, 10: -10.0, 11: 10.0}
Optimal Policy:  {1: 2, 2: 5, 5: 8, 8: 11, 3: 0, 4: 0, 6: 0, 7: 0, 9: 0, 10: 0, 11: 0}
